In [1]:

import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np

from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
import requests
import json
from xml.etree import ElementTree as ET
from statsmodels.stats.multitest import multipletests
#/data2/wangxuedong/mhc_test_data/similarity_region_pythonfiles/csvfiles_95/csv2.csv
df_csv2=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/find_region/pos_csv/tables5_part0.csv").values.tolist()

all_mean_arr=0.008131350120614151
all_std_arr=0.003374720423931657
mean_dict_different_race={'ACB': 0.043905219284227484,
 'ASW': 0.061747415774036245,
 'BEB': 0.042622549524204016,
 'CDX': 0.05629086218276262,
 'CEU': 0.03350899999290784,
 'CHB': 0.05201986349516006,
 'CHS': 0.03643899713354041,
 'CLM': 0.04224577966848677,
 'ESN': 0.03623741211034457,
 'FIN': 0.053129961026517944,
 'GBR': 0.05656052651694239,
 'GIH': 0.05157971151605565,
 'GWD': 0.03170015029229307,
 'IBS': 0.0371240314050134,
 'ITU': 0.050724822118831774,
 'JPT': 0.0517456479295656,
 'KHV': 0.04564560273080832,
 'LWK': 0.04973674801295715,
 'MSL': 0.04960771674064523,
 'MXL': 0.05377595974774834,
 'PEL': 0.045806236523102895,
 'PJL': 0.040244478916681585,
 'PUR': 0.04003893507415216,
 'STU': 0.048497323483452925,
 'TSI': 0.05007660537347943,
 'YRI': 0.031315221824936575}
std_dict_different_race={'ACB': 0.0035837115206611876,
 'ASW': 0.0044435636072078216,
 'BEB': 0.003260312673626174,
 'CDX': 0.00391015689396564,
 'CEU': 0.002766896630079425,
 'CHB': 0.0037059827211937194,
 'CHS': 0.002921456474220746,
 'CLM': 0.003272057962121683,
 'ESN': 0.0031803272910866064,
 'FIN': 0.0038008600285356704,
 'GBR': 0.003936807447491629,
 'GIH': 0.0037221796184063703,
 'GWD': 0.0029073393803639483,
 'IBS': 0.0029858040112478645,
 'ITU': 0.003597951675686167,
 'JPT': 0.003708599258289701,
 'KHV': 0.00343805410624286,
 'LWK': 0.003848934277722462,
 'MSL': 0.003934844393069718,
 'MXL': 0.003865357254429113,
 'PEL': 0.003392906748737866,
 'PJL': 0.003126523821241254,
 'PUR': 0.0031949284883587385,
 'STU': 0.00353413807778532,
 'TSI': 0.0036716230350532964,
 'YRI': 0.002910364879924007}
allrecords=[]
#in csv2
for record in df_csv2:

    if type(record[6])!=float:
        if record[1]=="All":
            if record[2]==0:#no mhc csvfiles                        
                position_of_gene=record[6]
                chr=position_of_gene.split(":")[0]
                start_position=int(position_of_gene.split(":")[1].split("-")[0])

                end_position=int(position_of_gene.split(":")[1].split("-")[1])
               
                if chr=="chr6":
                    filenames=os.listdir("/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/")
                    for filename in filenames:
                        if filename.startswith(chr+"_"):
                            df_split=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/"+filename)  

                            allrigion_P_top_1000=df_split[(df_split["Unnamed: 0"]>=start_position) & (df_split["Unnamed: 0"]<=end_position)]["P"].to_list()
                            if allrigion_P_top_1000:
                                mean_100_P=np.mean(allrigion_P_top_1000)
                                std_100_P=np.std(allrigion_P_top_1000, ddof=1)
                                pvalue=round((1-stats.norm.cdf(mean_100_P, all_mean_arr, all_std_arr)),15)
                                onerecord=[]
                                onerecord+=record[1:9]
                                onerecord+=[mean_100_P,std_100_P,pvalue]
                                allrecords.append(onerecord)
                                    

                            else:
                                continue
                        else:
                            continue
                else:
                 
                       
                    df_split=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/"+chr+".csv")  

                    allrigion_P_top_1000=df_split[(df_split["Unnamed: 0"]>=start_position) & (df_split["Unnamed: 0"]<=end_position)]["P"].to_list()
                    if allrigion_P_top_1000:
                        mean_100_P=np.mean(allrigion_P_top_1000)
                        std_100_P=np.std(allrigion_P_top_1000, ddof=1)
                        pvalue=round((1-stats.norm.cdf(mean_100_P, all_mean_arr, all_std_arr)),15)
                        onerecord=[]
                        onerecord+=record[1:9]
                        onerecord+=[mean_100_P,std_100_P,pvalue]
                        allrecords.append(onerecord)
                            

                    else:
                        continue
                    

            else:
                position_of_gene=record[6]
                start_position=int(position_of_gene.split(":")[1].split("-")[0])
                end_position=int(position_of_gene.split(":")[1].split("-")[1])
                df_mhc=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles/posmhc.csv")
                allrigion_P_top_1000=df_mhc[(df_mhc["Unnamed: 0"]>=start_position) & (df_mhc["Unnamed: 0"]<=end_position)]["P"].to_list()
                if allrigion_P_top_1000:
                    mean_100_P=np.mean(allrigion_P_top_1000)
                    std_100_P=np.std(allrigion_P_top_1000, ddof=1)
                    pvalue=round((1-stats.norm.cdf(mean_100_P, all_mean_arr, all_std_arr)),15)
                    onerecord=[]
                    onerecord+=record[1:9]
                    onerecord+=[mean_100_P,std_100_P,pvalue]
                    allrecords.append(onerecord)
                else:
                    continue

        else:#race!=all
            if record[2]==0:#no mhc
                position_of_gene=record[6]
                chr=position_of_gene.split(":")[0]
                start_position=int(position_of_gene.split(":")[1].split("-")[0])

                end_position=int(position_of_gene.split(":")[1].split("-")[1])
                if chr=="chr6":
                    filenames=os.listdir("/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/")
                    for filename in filenames:
                        if filename.startswith(chr+"_"):
                            df_split=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/"+filename)  
                        
                            allrigion_P_top_1000=df_split[(df_split["Unnamed: 0"]>=start_position) & (df_split["Unnamed: 0"]<=end_position)][record[1]].to_list()
                            if allrigion_P_top_1000:
                                mean_100_P=np.mean(allrigion_P_top_1000)
                                std_100_P=np.std(allrigion_P_top_1000, ddof=1)
                                pvalue=round((1-stats.norm.cdf(mean_100_P,mean_dict_different_race[record[1]],std_dict_different_race[record[1]])),15)
                                    
                                onerecord=[]
                                onerecord+=record[1:9]
                                onerecord+=[mean_100_P,std_100_P,pvalue]
                                allrecords.append(onerecord)
                            else:
                                continue
                        else:
                            continue
                else:
                   
                    df_split=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/"+chr+".csv")  
                
                    allrigion_P_top_1000=df_split[(df_split["Unnamed: 0"]>=start_position) & (df_split["Unnamed: 0"]<=end_position)][record[1]].to_list()
                    if allrigion_P_top_1000:
                        mean_100_P=np.mean(allrigion_P_top_1000)
                        std_100_P=np.std(allrigion_P_top_1000, ddof=1)
                        pvalue=round((1-stats.norm.cdf(mean_100_P,mean_dict_different_race[record[1]],std_dict_different_race[record[1]])),15)
                            
                        onerecord=[]
                        onerecord+=record[1:9]
                        onerecord+=[mean_100_P,std_100_P,pvalue]
                        allrecords.append(onerecord)
                    else:
                        continue
         
            else:#ismhc different race
                position_of_gene=record[6]
                start_position=int(position_of_gene.split(":")[1].split("-")[0])
                end_position=int(position_of_gene.split(":")[1].split("-")[1])
                df_mhc=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_race/mhc.csv")
                
                allrigion_P_top_1000=df_mhc[(df_mhc["Unnamed: 0"]>=start_position) & (df_mhc["Unnamed: 0"]<=end_position)][record[1]].to_list()
                if allrigion_P_top_1000:                
                    mean_100_P=np.mean(allrigion_P_top_1000)
                    std_100_P=np.std(allrigion_P_top_1000, ddof=1)
                    pvalue=round((1-stats.norm.cdf(mean_100_P,mean_dict_different_race[record[1]],std_dict_different_race[record[1]])),15)              
                    onerecord=[]
                    onerecord+=record[1:9]
                    onerecord+=[mean_100_P,std_100_P,pvalue]
                    allrecords.append(onerecord)
                else:
                    continue
    else:
        print("record",record)
        print("record6",record)
        onerecord=[]
        onerecord+=record[1:9]
        onerecord+=["null","null","null"]
        allrecords.append(onerecord)

#generatedf
def generatedf(columns,allrecords):
    dictforDF=dict()
    for i in range(len(columns)):
        midarr=[]
        for record in allrecords:           
            midarr.append(record[i])
        dictforDF[columns[i]]=midarr
    dataframe(dictforDF).to_csv("tables5_part1_nocorr.csv")
    return dataframe(dictforDF)


dfnew=generatedf(columns=["Race","isMHC","Position of disassortative mating region","Gene ID","Gene name","Position of gene","is_complement","Gene description","Average of P","Std of P","P-value"],allrecords=allrecords)


pvalue_list=dfnew["P-value"].tolist()
corr_pvals = multipletests(pvalue_list, method='fdr_bh')[1]
dfnew = dfnew.assign(corr_pvals=corr_pvals)
dfnew.to_csv("tables5_part1_withcorr.csv")


/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/wangxuedong/app/miniconda3/envs/mhc/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=ou